# 0. import libraries

In [1]:
!pip3 install --user sklearn
!pip install -U scikit-learn scipy matplotlib
!pip install seaborn

Requirement already up-to-date: scikit-learn in c:\users\308_2\.conda\envs\minji\lib\site-packages (0.23.1)
Requirement already up-to-date: scipy in c:\users\308_2\.conda\envs\minji\lib\site-packages (1.5.1)
Requirement already up-to-date: matplotlib in c:\users\308_2\.conda\envs\minji\lib\site-packages (3.3.0)


In [1]:
# import libraries
import json
import re # regular expression
import gc # to collect memory
from datetime import datetime

import tensorflow as tf
import pandas as pd
import numpy as np

# for NLP
# from konlpy.tag import Kkma
# from konlpy.utils import pprint
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
# import missingno as msno

plt.style.use('seaborn')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
font_name = matplotlib.font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
matplotlib.rc('font', family = font_name)

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

---
# 1. data

In [3]:
# load json files
with open("./res/genre_gn_all.json", encoding = 'utf-8') as json_file: genre_gn_all = json.load(json_file)
with open("./res/song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
with open("./res/train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
with open("./res/val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)

# json to dataframe
genre_gn_all = pd.Series(genre_gn_all).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
song_meta = pd.DataFrame(song_meta)
train = pd.DataFrame(train)
valid = pd.DataFrame(valid)

In [4]:
#대분류장르코드
# 장르코드 뒷자리 두 자리가 00인 코드를 필터링
gnr_code = genre_gn_all[genre_gn_all['gen_id'].str[-2:] == '00']

In [5]:
#상세장르코드
# 장르코드 뒷자리 두 자리가 00이 아닌 코드를 필터링
dtl_gnr_code = genre_gn_all[genre_gn_all['gen_id'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gen_id' : 'dtl_gen_id', 'genre' : 'dtl_genre'}, inplace = True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [7]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


# 2. 대분류에서 장르명과 일치하는 단어가 있는 태그를 찾기
---

---
## 1) tag dataframe 제작

In [8]:
# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag_map = train[['id', 'tags']]

# unnest tags
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

In [9]:
plylst_tag_map 

,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,까페
4,76951,잔잔한
...,...,...
476326,131982,퇴근길
476327,100389,노래추천
476328,100389,팝송추천
476329,100389,팝송


### 이건 그냥 플레이리스트 아이디와 맵핑된 코드였음.. 416331개가 아님

In [10]:
# 태그 별 매핑 빈도 수 저장 
tag_cnt = plylst_tag_map.groupby('tags').tags.count().reset_index(name = 'mapping_cnt')
tag_cnt['tags'] = tag_cnt['tags'].astype(str)
tag_cnt['mapping_cnt'] = tag_cnt['mapping_cnt'].astype(int)

In [11]:
tag_cnt.head(100)

,tags,mapping_cnt
0,,1
1,00,4
2,007,3
3,007시리즈,2
4,00s,3
5,00년,6
6,00년대,27
7,00년대노래,1
8,00년대발라드,2
9,00년도노래,1


In [12]:
tag_cnt.tail(100)

,tags,mapping_cnt
29060,힙플,2
29061,힙플라디오,1
29062,힙플레이스,1
29063,힙플페,3
29064,힙하게,1
29065,힙하고,1
29066,힙하고싶은날,1
29067,힙하다,4
29068,힙하우스,2
29069,힙한,209


In [13]:
tag_cnt

,tags,mapping_cnt
0,,1
1,00,4
2,007,3
3,007시리즈,2
4,00s,3
...,...,...
29155,힙해,5
29156,힙힙힙,2
29157,힛뎀포크,1
29158,힛뎀폭,1


### 태그 자체는 29159개였다!
전체 태그 저장해놓은 df: tag_cnt

---

## 2) 대분류에서 장르 name dataframe 만들기
- 유의사항
  - 중복 있음
  - 랩/힙합과 같이 /로 나눠진 것 존재
  - pop의 경우 팝이라는 단어로 더 많이 쓰임
---
- 중복 제거 필요
- / 있는 경우 나눠서 넣음
- 알앤비, 소울, rock, metal, 팝, country, 등등 단어 추가 (일단 생각나는게 이정도라서...)

In [14]:
gnr_code

,gen_id,genre
0,GN0100,발라드
6,GN0200,댄스
12,GN0300,랩/힙합
18,GN0400,R&B/Soul
22,GN0500,인디음악
32,GN0600,록/메탈
39,GN0700,성인가요
50,GN0800,포크/블루스
56,GN0900,POP
65,GN1000,록/메탈


In [15]:
genre_list=gnr_code[['genre']]

In [16]:
genre_list

,genre
0,발라드
6,댄스
12,랩/힙합
18,R&B/Soul
22,인디음악
32,록/메탈
39,성인가요
50,포크/블루스
56,POP
65,록/메탈


In [17]:
var_split = genre_list['genre'].str.split('/')
var_split


#value_name = var_split.str[0]

#2,3번째 항목에서 NaN 제거
value_name1 = var_split.str[1]
value_name1=value_name1[~value_name1.isnull()]
value_name2 = var_split.str[2]
value_name2=value_name2[~value_name2.isnull()]

#장르 리스트에서 /없애고, /로 나눈 항목 중 첫번째 것만 넣음
genre_list['genre'] = genre_list['genre'].str.split('/').str[0]

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
#object를 dataframe로
value_name1=pd.DataFrame(value_name1)
value_name1

,genre
12,힙합
18,Soul
32,메탈
50,블루스
65,메탈
90,힙합
100,Soul
109,블루스
198,태교


In [19]:
value_name2=pd.DataFrame(value_name2)
value_name2

,genre
109,컨트리


In [20]:
genre_list=pd.concat([genre_list, value_name1,value_name2], ignore_index=True)
genre_list

,genre
0,발라드
1,댄스
2,랩
3,R&B
4,인디음악
5,록
6,성인가요
7,포크
8,POP
9,록


In [21]:
genre_list=genre_list.drop_duplicates() #중복 제거

#index reset하겠음(문자 단위 검색 목적이므로)
genre_list=genre_list.reset_index()
del genre_list['index']

genre_list

,genre
0,발라드
1,댄스
2,랩
3,R&B
4,인디음악
5,록
6,성인가요
7,포크
8,POP
9,일렉트로니카


In [22]:
#키워드 추가
#dance, 알앤비, rock, 인디, 성인, 팝, 일렉트로닉, 키즈, 종교, idol, 소울, 일본(12개)
genre_list=genre_list.append([{'genre':'dance'},{'genre':'알앤비'},{'genre':'rock'},{'genre':'인디'},{'genre':'성인'},
                           {'genre':'팝'},{'genre':'일렉트로닉'},{'genre':'키즈'},{'genre':'종교'},{'genre':'idol'},
                              {'genre':'소울'},{'genre':'일본'}]
                          ,ignore_index=True)
genre_list

,genre
0,발라드
1,댄스
2,랩
3,R&B
4,인디음악
5,록
6,성인가요
7,포크
8,POP
9,일렉트로니카


---

## 3) 장르와 연관된 태그
tag_cnt: 태그 모음  
genre_list: 장르 모음

- 문자열 검색시 contains는 일부 포함된것도 검색
- isin은 완벽하게 일치인듯?

In [23]:
#'발라드'
print(genre_list['genre'][0])
test=tag_cnt[tag_cnt['tags'].str.contains(genre_list['genre'][0])] 
test=test.reset_index()
del test['index']
test

발라드


,tags,mapping_cnt
0,00년대발라드,2
1,1980년대발라드,1
2,1990년대발라드,2
3,2000년대발라드,6
4,7080발라드,1
...,...,...
114,한국발라드,6
115,한국형발라드,1
116,혼성발라드,1
117,힙발라드,4


In [24]:
#'댄스'
print(genre_list['genre'][1])
test=tag_cnt[tag_cnt['tags'].str.contains(genre_list['genre'][1])] 
test=test.reset_index()
del test['index']
test

댄스


,tags,mapping_cnt
0,2010년대댄스,1
1,30대댄스가요,1
2,8090댄스,1
3,80년대댄스팝,1
4,90년대_댄스,1
...,...,...
135,하우스_댄스,1
136,헬스장에서듣는댄스곡,1
137,혼성댄스그룹,1
138,힐링댄스,1


### 전체 출력(반복문)

In [25]:
for i in range(0,43):
    print("index %d: " %i,end='')
    print(genre_list['genre'][i]+'\n')
    test=tag_cnt[tag_cnt['tags'].str.contains(genre_list['genre'][i])] 
    test=test.reset_index()
    del test['index']
    print(test)
    print("\n---------------------------------------------------------\n")

index 0: 발라드

          tags  mapping_cnt
0      00년대발라드            2
1    1980년대발라드            1
2    1990년대발라드            2
3    2000년대발라드            6
4      7080발라드            1
..         ...          ...
114      한국발라드            6
115     한국형발라드            1
116      혼성발라드            1
117       힙발라드            4
118      힙합발라드            1

[119 rows x 2 columns]

---------------------------------------------------------

index 1: 댄스

           tags  mapping_cnt
0      2010년대댄스            1
1       30대댄스가요            1
2        8090댄스            1
3       80년대댄스팝            1
4       90년대_댄스            1
..          ...          ...
135      하우스_댄스            1
136  헬스장에서듣는댄스곡            1
137      혼성댄스그룹            1
138        힐링댄스            1
139        힙합댄스            4

[140 rows x 2 columns]

---------------------------------------------------------

index 2: 랩

         tags  mapping_cnt
0        CCM랩            1
1         감성랩           14
2       감성적인랩            1
3

          tags  mapping_cnt
0       감성뉴에이지            1
1       겨울뉴에이지            1
2     귀르가즘뉴에이지            1
3      나른한뉴에이지            7
4         뉴에이지         3493
5        뉴에이지곡            6
6        뉴에이지로            1
7       뉴에이지모음            4
8      뉴에이지발라드            1
9      뉴에이지스테디            1
10     뉴에이지실내악            1
11     뉴에이지연주곡            1
12      뉴에이지음악            8
13     뉴에이지자장가            2
14      뉴에이지최애            1
15     뉴에이지클래식            1
16      뉴에이지탑백            1
17     뉴에이지피아노           49
18  뉴에이지피아노연주곡            3
19      뉴에이지힐링           10
20   드라마뉴에이지음악            1
21   마음에좋은뉴에이지            1
22       밤뉴에이지            1
23    부드러운뉴에이지            6
24      아침뉴에이지            1
25    템포빠른뉴에이지            1
26    피아노_뉴에이지            1
27     피아노뉴에이지            2

---------------------------------------------------------

index 14: J-POP

Empty DataFrame
Columns: [tags, mapping_cnt]
Index: []

-------------------------------------------------------

       tags  mapping_cnt
0     국외뮤지컬            3
1       뮤지컬           84
2    뮤지컬OST           12
3   뮤지컬_OST            1
4     뮤지컬넘버           14
5     뮤지컬노래            7
6     뮤지컬들의            1
7     뮤지컬배우            1
8     뮤지컬영화            8
9     뮤지컬음악            3
10   뮤지컬풍노래            1
11    초기뮤지컬            1
12  하이스쿨뮤지컬            1
13    해외뮤지컬            2

---------------------------------------------------------

index 24: 크리스마스

            tags  mapping_cnt
0   더매직오브크리스마스타임            1
1       라스트크리스마스            1
2       로맨틱크리스마스            1
3        메리크리스마스           50
4      미리듣는크리스마스            2
5      미리메리크리스마스            9
6        솔로크리스마스            1
7       아이돌크리스마스            1
8        여름크리스마스            1
9     올해도메리크리스마스            1
10     외로운_크리스마스            1
11     재즈보컬크리스마스            1
12      재즈와크리스마스            1
13       재즈크리스마스            1
14         크리스마스          948
15     크리스마스2019            1
16     크리스마스_전야제            1
17      

               tags  mapping_cnt
0          Hardrock            5
1   alternativerock            2
2           artrock            2
3       britishrock            2
4      brockhampton            1
5       classicrock            1
6             drock            1
7          glamrock            2
8             krock            3
9        modernrock            1
10        oneokrock            2
11      paintitrock            1
12        punk_rock            1
13      rock_n_roll            3
14      rockandroll            4
15         rockband            3
16    rockisnotdead            1
17     rocktheworld            1
18        rock이그리울때            1
19           rock추천            1
20        rootsrock            2
21      seattlerock            1

---------------------------------------------------------

index 34: 인디

             tags  mapping_cnt
0        2010년대인디            2
1    2월Yasisi인디뮤직            1
2             k인디            1
3        가을과인디의만남            1
4           

---
---
# <span style="color:red">------------7/16ㄱ</span>
---

# 3. 플레이리스트 이름과 장르/태그 비교

## 1) train의 플레이리스트 타이틀 중 장르 이름이 들어간 플레이리스트 찾기


In [26]:
for i in range(0,43):
    print("index %d: " %i,end='')
    print(genre_list['genre'][i]+'\n')
    test=train[train['plylst_title'].str.contains(genre_list['genre'][i])] 
    test=test.reset_index()
    del test['index']
    print(test['plylst_title'])
    print("\n---------------------------------------------------------\n")

index 0: 발라드

0                 발라드로 감성을 발라드림(슬픔주의)
1                       걸그룹 발라드 명곡 모음
2       비오는날 감성 자극 듣기 좋은 어쿠스틱/피아노 발라드
3       "이별 후에 들으면 더 슬픈 감성 발라드 인디 노래"
4       고음병걸린 당신을위한 명품발라드100선　　#여자가수편
                    ...              
3547             한 번 쯤 스쳐지나간 멜로디의 발라드
3548                 출근할때 듣는 시원한 롹발라드
3549              좋은발라드 다 모아놓고 한번에 듣기
3550                        ★슬픈감성발라드★
3551                         취향저격 발라드
Name: plylst_title, Length: 3552, dtype: object

---------------------------------------------------------

index 1: 댄스

0                  졸음운전방지※댄스댄스한 플레이리스트♥
1                운동 중에 들으면 힘나는 빠른 댄스곡!!
2       정신차려보면 이미 댄스 무아지경! 2000년대초 댄스음악
3                        po내적댄스wer 팝송모음
4             댄스 팝 틀어놓고 신나게 춤추며 스트레스 풀자
                     ...               
1262            무더운 여름을 날려주는 걸그룹 댄스곡^_^
1263                  90년대 인기곡(댄스곡 모음집)
1264                호텔 라운지 느낌 일렉트로 댄스뮤직
1265                    댄스그룹이 부른 감미로운 곡
1266                        힙합과 댄스가 만나면
Name: p

0                     J-POP prism♪
1                   듣기 좋은 J-POP 모음
2      J-POP의 매력을 모르는 사람에게 들려주고싶다!
3              쟈니스 없는 J-POP 베스트 12
4          J-POP더위에 지칠땐 댄스와 일렉팝!!!
                  ...             
143             '봄'을 노래한 J-POP 디바들
144                  ★★완전좋은J-POP★★
145                   영원한 J-POP 우상
146     00년대 즐겨듣던 J-POP (시부야케이 위주)
147                  J-POP의 명곡들...
Name: plylst_title, Length: 148, dtype: object

---------------------------------------------------------

index 15: 월드뮤직

0                   진한 커피향과 짙은 감성의 월드뮤직
1                  점심 식사시간용 월드뮤직. 주로 라틴
2                     흠.. 월드뮤직이 어렵다구요?!
3                         팝과 록에 담긴 월드뮤직
4                ♠이국적인 여유로운 일상,재즈&월드뮤직♠
5                 비오는날 감성적인 월드뮤직과 함께...
6                 해변 생각나는 이국적인 사운드 월드뮤직
7                 여행을 떠나고싶은 감성! 월드뮤직 모음
8                        ♣내 입맛돋구는 월드뮤직♣
9           언더커버뮤직 51회: 월드뮤직 어디까지 들어봤니?
10                  명곡 팝,월드뮤직,힙합 매장음악모음
11           재즈, 클래식, 월드뮤직 소수와 공유하는 음악들
12          

0            뮤직테라피 스테디
1    요가 명상과 함께하는 뮤직테라피
2     마음을 안정시켜주는 뮤직테라피
3    따듯한 봄날에 즐기는 뮤직테라피
Name: plylst_title, dtype: object

---------------------------------------------------------

index 23: 뮤지컬

0                내가 좋아하는 뮤지컬 영화 노래들~
1      디스코, 힙합이 함께하는 뮤지컬 드라마, 더 겟 다운
2              뮤지컬의 재해석! 또 다른 느낌의 명곡
3           뮤덕이 추천하는 뮤지컬 넘버._. (한국어)
4               디즈니 음악과 뮤지컬영화 음악이 함께
                   ...              
129                     매력적인 뮤지컬 넘버들
130                           뮤지컬 뮤직
131               듣기 좋은 뮤지컬 노래 추천 모음
132                   기억해야 할 뮤지컬 넘버들
133     뮤지컬영화.공연.애니 취향존중 곡 선정 (계속업뎃)
Name: plylst_title, Length: 134, dtype: object

---------------------------------------------------------

index 24: 크리스마스

0                         크리스마스 분위기에 흠뻑 취하고 싶을때
1                              크리스마스를 책임질 캐롤 모음
2                                  느낌있는 크리스마스 쏭
3                                        크리스마스9
4                                   미리 메리 크리스마스
                         ...

0                       트렌디하고 그루브한 힙합/알앤비 MUSIC
1                             욕 나올정도로 좋은 해외 알앤비
2                        가을 밤과 어울리는 감성보컬 알앤비 모음
3                               커피와 찰떡궁합 알앤비 힙합
4                                   아름다운 알앤비&힙합
                         ...                   
687                       세련된 그루브! 어반 알앤비 & 힙합!
688             (수시 업뎃) 홍대 빈티지샵 재질의 트렌디한 국힙/알앤비
689                        멜로디가좋고 듣기좋은 알앤비 힙합!!
690                       비오는 밤, 그루브 넘치는 알앤비 팝~
691    1-알앤비 I 감성 터지는 늦은 밤, 혼자만의 생각에 잠기기 좋은 20곡
Name: plylst_title, Length: 692, dtype: object

---------------------------------------------------------

index 33: rock

0                              괜찬은 rock! 예~
1                                 haya rock
2                J rock 뉴비들을 위한 입문하기 좋은 노래들
3                           락힙합 rock hiphop
4                                dear. rock
5          Thanks God It’s rock Festival !!
6               드라이브할때 듣는 파워풀한 베이스의 해외 rock
7                        나의 Modern rock..

0                               보아일본곡
1                     일본 노래 모음 2000년대
2                     트와이스 일본 타이틀곡 모음
3        지금 이순간 일본여행의 추억과 사랑이 떠오를때...
4                    주목 받는 힙한 일본 인디음악
                    ...              
187    인싸들의 잇템 씨티팝! 일본에만 있니? 한국에도 많어!
188                내가 봤던 인생애니 ost(일본)
189            그들만의 개성이 느껴지는 일본의 ROCK
190             폴 매카트니 일본 공연 set list
191               일본어라고 놓치면 후회할 띵곡 모음
Name: plylst_title, Length: 192, dtype: object

---------------------------------------------------------



오히려 플레이리스트 이름으로 한게 예상보다 좋은 결과가 나온 것 같다.

- 키즈 라는 단어는 빼야할듯? (먼데이키즈, 스트레이키즈 가 섞여있음)

---

## 2) train의 플레이리스트 타이틀 중 태그와 일치하는 이름이 있는 플레이리스트 찾기

In [27]:
for i in range(0,50):#2만개인데 10분째 다 안나옴..
    print("index %d: " %i,end='')
    print(tag_cnt['tags'][i]+'\n')
    test=train[train['plylst_title'].str.contains(tag_cnt['tags'][i])] 
    test=test.reset_index()
    del test['index']
    print(test['plylst_title'])
    print("\n---------------------------------------------------------\n")

index 0: 

0                                   여행같은 음악
1                                   요즘 너 말야
2                     편하게, 잔잔하게 들을 수 있는 곡.-
3                     크리스마스 분위기에 흠뻑 취하고 싶을때
4                                  추억의 노래 ㅋ
                        ...                
115066                        METAL E'SM #2
115067       빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!
115068                #1. 눈물이 앞을 가리는 나의_이야기
115069    퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP
115070                  FAVORITE POPSONG!!!
Name: plylst_title, Length: 115071, dtype: object

---------------------------------------------------------

index 1: 00

0                           100번 들어도 기분좋은 가요 !
1                고음병걸린 당신을위한 명품발라드100선　　#여자가수편
2                          4월 12일자 빌보드 Hot 100
3              정신차려보면 이미 댄스 무아지경! 2000년대초 댄스음악
4                            몽환러버의 ‘감성 2:00AM’
                         ...                  
1903             '00:00~~06:00' 가사를 마음에 담는 시간.
1904                         오늘의 추천곡 10선 - 005
1905  

0                                  140416
1    ARIA SinglesChart Top20 170416(호주차트)
2                       0416. 세월호를 추모하며..
3                        아직, 있다. 20140416
4                           Remember 0416
5                          Remember140416
6                  20170416 콜드플레이 내한 셋리스트
Name: plylst_title, dtype: object

---------------------------------------------------------

index 19: 0421

0                                        0421
1    Official SinglesChart Top20 170421(영국차트)
Name: plylst_title, dtype: object

---------------------------------------------------------

index 20: 04_29

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 21: 05

0                                푸른밤의 플레이리스트 05
1                                10:05 너와 나의 시간
2      아무로 나미에 [Space of Hip-Pop 2005] Set List
3          ARIA SinglesChart Top20 160508(호주차트)
4                       요즘 많이 듣는 곡 (2019.05.09)
                         ...     

0    추1000000
Name: plylst_title, dtype: object

---------------------------------------------------------

index 37: 1000곡

0                      기억 저편 해마 까지 털어 모은 1000곡
1                              1000곡이담긴 플레이리스트
2           [외힙] 찐 흑인음악 1000곡 채우기 Vol.1_상시업데이트
3         (2탄) 나 혼자 듣자고 모은 팝송! 1000곡! 모으는 중...
4                                      도전1000곡
5              90년대 태어난 사람들은 다 아는 신나는 노래 1000곡
6                       잔잔하게 듣기 좋은 재즈 1000곡 모음
7                             무작위 선곡한 노래 1000곡
8                  (1탄) 나 혼자 듣자고 모은 팝송! 1000곡!
9     ※집중력최고※ 카페에 앉아 조용한 볼륨, 인디1000곡 들으며 작업하자!
10                                    1000곡 텟트
11    ❤따스한 봄날이 가기전에 1000곡 다 들을때까지 산책하기~ 도저언~!❤
Name: plylst_title, dtype: object

---------------------------------------------------------

index 38: 1000만관객동원

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 39: 1000칼로리

Series([], Name: plylst_title, dtype: object)

------------------------

0                                 안예은 BEST 10곡
1                      1)슬픈 이별, R&B/발라드 노래 10곡
2                             여름밤 듣기 좋은 음악 10곡
3                             지금 듣기 좋은 발라드 10곡
4                                 요즘 많이 듣는 10곡
5     용기,고민,이별의 감정을 이야기처럼 풀어내는 10곡(상세정보 꼭 확인!)
6                    새벽녘 우리의 마음을 어루만져주는 음악 10곡
7                                딱 10곡만 들어주세요.
8             밤에 조용히 들을만한  목소리가 좋은 여성보컬 노래 10곡
9                                 롱디커플을 위한 10곡
10                            STAY라는 제목의 명곡10곡
11       샤워할때 들으면 수도세 폭발해서 엄마한태 등짝맞는 힙한 노래 10곡
12                                클럽 일렉 음악 10곡
13                           내 마음을 읽어주는 노래 10곡
14                    매장용 10곡! 골라듣자 (국내5,해외5곡)
15                           ccm베스트10곡을 모아봤어요.
16                               제일 신나는 국힙 10곡
17               (vol.1~7 총집합) 시끄럽지 않은 일렉 210곡
18                    감성힙합? 힐링타임? 힙합 10곡 들어가요~
19                                외국영화 ost 10곡
20                    즐거운 일요일^^[최신곡,추천곡 모음10곡]
21           

#### 너무 많아서 끊었다!!
- 그리고 가수이름같은 경우 겹치는게 많음 -> 이건 태그로 안하고 가수 이름과 플레이리스트 대조하는게 좋을듯하다
- 태그중에 쓸데없는것도 많음 
    - 7777 이런거...(뭔지모름)
- 의도가 다른것도 있음
    - 101 -> 프로듀스 101, 오늘의 음악 161101, 유희열의 스케치북 101회 이런것도 다 들어감
    
#### 이건 안쓰는게 나을 것 같다.

## 3) train의 플레이리스트 타이틀 중 가수 이름과 일치하는 이름이 있는 플레이리스트 찾기

---

In [28]:
#아티스트 id별 이름 매칭

In [29]:
#담을 곳 선언
artist_list=pd.DataFrame()

#artist id와 name
#song_meta['artist_id_basket']
#song_meta['artist_name_basket']

# 첫 번째것만 담겠다. 왜냐하면 나중에 딕셔너리는 정렬이 안됨ㅠ
artist_list['id']=song_meta['artist_id_basket'].str[0] #list 첫번째 항목만 넣는다
artist_list['name']=song_meta['artist_name_basket'].str[0] #list 첫번째 항목만 넣는다

In [30]:
artist_list

,id,name
0,2727,Various Artists
1,29966,Murray Perahia
2,3361,Peter Gabriel
3,838543,Matoma
4,560160,Jude Law
...,...,...
707984,166499,Fela Kuti
707985,11837,Cyndi Lauper
707986,437,윤종신
707987,729868,Nature Piano


In [31]:
# artist id중 맨 앞에 있는 이름으로 정렬
artist_list=artist_list.sort_values(by=['id'],axis=0)
artist_list

,id,name
305875,1,나훈아
169859,1,나훈아
321108,1,나훈아
48843,1,나훈아
170749,1,나훈아
...,...,...
512947,2865590,라포
334050,2865591,펭수
442813,2866236,Midas Touch
318592,2866406,Moon Arch


In [32]:
artist_list=artist_list.drop_duplicates('id', keep="first")

#index reset하겠음(문자 단위 검색 목적이므로)
artist_list=artist_list.reset_index()
del artist_list['index']

artist_list

,id,name
0,1,나훈아
1,2,남화용
2,5,유현상
3,6,김학래
4,7,정태춘
...,...,...
103573,2865590,라포
103574,2865591,펭수
103575,2866236,Midas Touch
103576,2866406,Moon Arch


- id 3이나 4가 사라진게아니라 sont_meta data 내에는 없다..

In [33]:
for i in range(0,100):#10만개는 너무많은듯.. 일단 일부만 뽑아봄
    print("index %d: " %i,end='')
    print(artist_list['name'][i]+'\n')
    test=train[train['plylst_title'].str.contains(artist_list['name'][i])] 
    test=test.reset_index()
    del test['index']
    print(test['plylst_title'])
    print("\n---------------------------------------------------------\n")

index 0: 나훈아

0          나훈아 히트곡 모음
1    영원한 트로트의 황제! 나훈아
2      트로트 꺽기의 황제 나훈아
Name: plylst_title, dtype: object

---------------------------------------------------------

index 1: 남화용

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 2: 유현상

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 3: 김학래

0    불후의 명곡 전설을 노래하다...YB&김학래&김광석 (편)
Name: plylst_title, dtype: object

---------------------------------------------------------

index 4: 정태춘

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 5: 김태곤

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 6: 이치현

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 7: 임창제

Series([], Name: plylst_title, dtype: object)

-------------------

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 42: 송대관

0    송대관 태진아 설운도 현철 히트곡 모음
1            송대관 사랑이무어냐 모음
2             송대관 히트곡모음 1집
Name: plylst_title, dtype: object

---------------------------------------------------------

index 43: 강권순

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 44: 프로젝트 살

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 45: 김덕수

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 46: 김매자

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 47: 김명덕

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 48: 김뻑국

Series([], Name: plylst_title, dtype: object)

----------------------------------

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------

index 99: 윤형주

Series([], Name: plylst_title, dtype: object)

---------------------------------------------------------



#### 쓸만한듯?

### <span style="color:#00c3ff;">어떻게 쓸 것인가?</span>
    
- train data에 개수 추가? ->의미없을듯
- 플레이리스트 타이틀 중 장르 이름이 들어간 플레이리스트 찾기
    - **장르와 노래 연결**해서 추가 작업 필요
- 플레이리스트 타이틀 중 가수 이름과 일치하는 이름이 있는 플레이리스트 찾기
    - **가수별 노래 list 필요**
- **dataframe으로 만들어야할듯?** 아니면 이 dataframe에서 검색해서 index를 뽑음->그다음 검색하는 방법으로 사용해야할 것 같다.

---


# <span style="color:red">----0718, 19 추가내용ㄱ</span>

# train의 song에서 가장 많은 장르를 뽑는다.

In [34]:
train_gen=train

In [35]:
train_gen

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [36]:
# 플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map = train[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest


In [37]:
# 곡 아이디(id)와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

# unnest song_gn_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)


# unnested 데이터프레임 생성 : song_gnr_map
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

# unnest 객체 제거
del song_gnr_map_unnest

#---장르

# 1-1. join 목적으로 곡 아이디 integer to string 변환
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# 1-2. 플레이리스트-곡 테이블 plylst_song_map + 곡-장르 테이블 song_gnr_map join
plylst_song_gnr_map = pd.merge(plylst_song_map, song_gnr_map, how = 'left', left_on = 'songs', right_on = 'song_id')

# 2. 플레이리스트 별 곡 장르 수 count 테이블 생성 : plylst_song_gnr_cnt
plylst_song_gnr_cnt = pd.DataFrame(plylst_song_gnr_map.groupby('id').gnr_code.nunique())

In [38]:
plylst_song_gnr_cnt

,gnr_code
id,
1,2
1000,10
100000,6
100001,13
100002,6
...,...
99991,8
99992,3
99993,3


In [39]:
plylst_song_gnr_cnt=plylst_song_gnr_cnt.reset_index()
plylst_song_gnr_cnt

,id,gnr_code
0,1,2
1,1000,10
2,100000,6
3,100001,13
4,100002,6
...,...,...
115066,99991,8
115067,99992,3
115068,99993,3
115069,99997,4


오름차순으로 바꿔도 똑같음. 뭔가 정렬기준이 이상해진듯  
115071이 train인거는 맞는데.. id가 string인듯??

In [40]:
plylst_song_gnr_cnt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        115071 non-null  object
 1   gnr_code  115071 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ MB


object 형식으로 되어있는 것을 확인할 수 있다.

In [41]:
# id를 int형으로 변환
plylst_song_gnr_cnt['id']=pd.to_numeric(plylst_song_gnr_cnt['id'])
plylst_song_gnr_cnt

,id,gnr_code
0,1,2
1,1000,10
2,100000,6
3,100001,13
4,100002,6
...,...,...
115066,99991,8
115067,99992,3
115068,99993,3
115069,99997,4


In [42]:
plylst_song_gnr_cnt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   id        115071 non-null  int64
 1   gnr_code  115071 non-null  int64
dtypes: int64(2)
memory usage: 1.8 MB


In [43]:
#정렬
# 장르 제일 적은거 0개 (id 128219), 가장 많은거 26개
#plylst_song_gnr_cnt=plylst_song_gnr_cnt.sort_values(by=['gnr_code'],axis=0)
plylst_song_gnr_cnt=plylst_song_gnr_cnt.sort_values(by=['id'],axis=0)
plylst_song_gnr_cnt

,id,gnr_code
0,1,2
48395,2,8
65067,4,8
73290,5,10
81634,6,5
...,...,...
44566,153423,11
44567,153425,4
44568,153426,5
44569,153427,5


In [44]:
plylst_song_gnr_map

,id,songs,song_id,gnr_code
0,61281,525514,525514,GN1400
1,61281,129701,129701,GN0900
2,61281,129701,129701,GN1000
3,61281,383374,383374,GN1000
4,61281,562083,562083,GN0900
...,...,...,...,...
6837821,100389,51373,51373,GN1300
6837822,100389,640239,640239,GN1500
6837823,100389,13759,13759,GN0900
6837824,100389,13759,13759,GN1000


In [45]:
gnr_count=pd.DataFrame()
#playlist id를 행으로 추가하기
gnr_count['plylst_id']=plylst_song_gnr_cnt['id']
gnr_count=gnr_count.reset_index()
del gnr_count['index']

In [46]:
gnr_count

,plylst_id
0,1
1,2
2,4
3,5
4,6
...,...
115066,153423
115067,153425
115068,153426
115069,153427


In [47]:
#빈 열 만들기
gnr_count['most_appearing_gnr']=pd.Series()

C:\Users\308_2\.conda\envs\minji\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [48]:
gnr_count

,plylst_id,most_appearing_gnr
0,1,NaN
1,2,NaN
2,4,NaN
3,5,NaN
4,6,NaN
...,...,...
115066,153423,NaN
115067,153425,NaN
115068,153426,NaN
115069,153427,NaN


In [49]:
plylst_song_gnr_map[plylst_song_gnr_map['id']=='1']

,id,songs,song_id,gnr_code
4583014,1,47805,47805,GN0300
4583015,1,308020,308020,GN0300
4583016,1,662131,662131,GN0300
4583017,1,418970,418970,GN0500
4583018,1,418970,418970,GN0300
4583019,1,117747,117747,GN0500
4583020,1,117747,117747,GN0300


In [50]:
# test, 임의로 '1'이라는 playlist id와 매치
plylst_song_gnr_map[plylst_song_gnr_map['id']=='1'].describe()

,id,songs,song_id,gnr_code
count,7,7,7,7
unique,1,5,5,2
top,1,418970,418970,GN0300
freq,7,2,2,5


plylst_song_gnr_map['id']가 object여야한다.  
만약에 plylst_song_gnr_map['id']가 정수형인 경우 평균, 최댓값 등이 구해져버린다.   
　　　 id  
count	7.0  
mean	1.0  
std	0.0  
min	1.0  
25%	1.0  
50%	1.0  
75%	1.0  
max	1.0  
이런식으로 나옴

In [51]:
plylst_song_gnr_map[plylst_song_gnr_map['id']=='1'].describe()['gnr_code']['top']

'GN0300'

In [52]:
testmap=pd.DataFrame()
testmap=gnr_count
testmap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   plylst_id           115071 non-null  int64  
 1   most_appearing_gnr  0 non-null       float64
dtypes: float64(1), int64(1)
memory usage: 1.8 MB


In [53]:
# testmap에서 plylst_id의 type가 int64라서 object로 바꿔주겠음
testmap['plylst_id']=testmap['plylst_id'].astype(str)
testmap['most_appearing_gnr']=testmap['most_appearing_gnr'].astype(str)
testmap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   plylst_id           115071 non-null  object
 1   most_appearing_gnr  115071 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


In [54]:
type(testmap['plylst_id'][1])

str

In [55]:
# test, 임의로 '1'이라는 playlist id와 매치
testmap['plylst_id'][1]

'2'

In [56]:
plylst_song_gnr_map[plylst_song_gnr_map['id']==testmap['plylst_id'][1]].describe()['gnr_code']['top']

'GN0500'

In [57]:
plylst_song_gnr_map['id']

0           61281
1           61281
2           61281
3           61281
4           61281
            ...  
6837821    100389
6837822    100389
6837823    100389
6837824    100389
6837825    100389
Name: id, Length: 6837826, dtype: object

In [58]:
testmap['plylst_id']

0              1
1              2
2              4
3              5
4              6
           ...  
115066    153423
115067    153425
115068    153426
115069    153427
115070    153428
Name: plylst_id, Length: 115071, dtype: object

In [59]:
testmap

,plylst_id,most_appearing_gnr
0,1,nan
1,2,nan
2,4,nan
3,5,nan
4,6,nan
...,...,...
115066,153423,nan
115067,153425,nan
115068,153426,nan
115069,153427,nan


In [60]:
plylst_song_gnr_map[plylst_song_gnr_map['id']==testmap['plylst_id'][0]]

,id,songs,song_id,gnr_code
4583014,1,47805,47805,GN0300
4583015,1,308020,308020,GN0300
4583016,1,662131,662131,GN0300
4583017,1,418970,418970,GN0500
4583018,1,418970,418970,GN0300
4583019,1,117747,117747,GN0500
4583020,1,117747,117747,GN0300


In [61]:
train_topgnr=pd.DataFrame()
train_topgnr=train

#빈 열 만들기
train_topgnr['most_appearing_gnr']=pd.Series()

train_topgnr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tags                115071 non-null  object 
 1   id                  115071 non-null  int64  
 2   plylst_title        115071 non-null  object 
 3   songs               115071 non-null  object 
 4   like_cnt            115071 non-null  int64  
 5   updt_date           115071 non-null  object 
 6   most_appearing_gnr  0 non-null       float64
dtypes: float64(1), int64(2), object(4)
memory usage: 6.1+ MB


C:\Users\308_2\.conda\envs\minji\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [62]:
#id를 object로
train_topgnr['id']=train_topgnr['id'].astype(str)
train_topgnr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tags                115071 non-null  object 
 1   id                  115071 non-null  object 
 2   plylst_title        115071 non-null  object 
 3   songs               115071 non-null  object 
 4   like_cnt            115071 non-null  int64  
 5   updt_date           115071 non-null  object 
 6   most_appearing_gnr  0 non-null       float64
dtypes: float64(1), int64(1), object(5)
memory usage: 6.1+ MB


In [68]:
import time
start = time.time()

for i in range(0,115071):
    train_topgnr['most_appearing_gnr'][i]=plylst_song_gnr_map[plylst_song_gnr_map['id']==train_topgnr['id'][i]].describe()['gnr_code']['top']
    if(i%100==0):
        print(i)
        print("time :", time.time() - start) 
        print("\n")
        
print("FINISH TIME!! :", time.time() - start) 

C:\Users\308_2\.conda\envs\minji\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0
time : 0.3001976013183594


100
time : 27.987226963043213


200
time : 55.47468423843384


300
time : 83.0281355381012


400
time : 110.24134922027588


500
time : 137.27557682991028


600
time : 163.3952717781067


700
time : 189.41136121749878


800
time : 215.3234522342682


900
time : 241.49001026153564


1000
time : 267.5238838195801


1100
time : 293.5658736228943


1200
time : 319.58270049095154


1300
time : 345.55178785324097


1400
time : 371.85097074508667


1500
time : 398.08627223968506


1600
time : 424.24055576324463


1700
time : 450.2106010913849


1800
time : 476.1919026374817


1900
time : 502.19788694381714


2000
time : 528.3695523738861


2100
time : 554.3413982391357


2200
time : 580.3891687393188


2300
time : 606.4554281234741


2400
time : 632.6229748725891


2500
time : 658.6038556098938


2600
time : 684.5321106910706


2700
time : 710.4982266426086


2800
time : 736.5041697025299


2900
time : 762.5323736667633


3000
time : 788.3984415531158


3100
time

25000
time : 6511.582785606384


25100
time : 6537.513469219208


25200
time : 6563.485305786133


25300
time : 6589.419936418533


25400
time : 6615.353005170822


25500
time : 6641.363479375839


25600
time : 6667.537497520447


25700
time : 6693.4232614040375


25800
time : 6719.403518199921


25900
time : 6745.463989496231


26000
time : 6771.519901752472


26100
time : 6797.598807096481


26200
time : 6823.664920568466


26300
time : 6849.686430454254


26400
time : 6875.749516248703


26500
time : 6901.832934856415


26600
time : 6928.05692076683


26700
time : 6954.011932373047


26800
time : 6979.911998271942


26900
time : 7005.9020318984985


27000
time : 7031.844758749008


27100
time : 7057.81102514267


27200
time : 7083.777188301086


27300
time : 7109.854519367218


27400
time : 7135.949620723724


27500
time : 7161.882449626923


27600
time : 7188.066106557846


27700
time : 7213.889626979828


27800
time : 7239.87477350235


27900
time : 7265.816854476929


28000
time 

49600
time : 12912.495484828949


49700
time : 12938.551846504211


49800
time : 12964.654972314835


49900
time : 12990.551581859589


50000
time : 13016.576808691025


50100
time : 13042.518228054047


50200
time : 13068.420812368393


50300
time : 13094.399705410004


50400
time : 13120.503061532974


50500
time : 13146.405991315842


50600
time : 13172.275433301926


50700
time : 13198.237508058548


50800
time : 13224.392158031464


50900
time : 13250.490077257156


51000
time : 13276.392463445663


51100
time : 13302.324224948883


51200
time : 13328.21367764473


51300
time : 13354.095245361328


51400
time : 13380.037443876266


51500
time : 13406.071496963501


51600
time : 13432.103264808655


51700
time : 13458.199197292328


51800
time : 13484.588724136353


51900
time : 13510.621329307556


52000
time : 13536.62546300888


52100
time : 13562.626073122025


52200
time : 13588.64357829094


52300
time : 13614.65152144432


52400
time : 13640.628007888794


52500
time : 13666

73900
time : 19236.09059023857


74000
time : 19262.398803949356


74100
time : 19288.343601465225


74200
time : 19314.452502012253


74300
time : 19340.49091243744


74400
time : 19366.497056007385


74500
time : 19392.494176387787


74600
time : 19418.588562726974


74700
time : 19444.625545024872


74800
time : 19470.55108189583


74900
time : 19496.57565522194


75000
time : 19522.753247737885


75100
time : 19548.701749563217


75200
time : 19574.6196269989


75300
time : 19600.56655240059


75400
time : 19626.47149538994


75500
time : 19652.418969154358


75600
time : 19678.44869661331


75700
time : 19704.5440762043


75800
time : 19730.645025491714


75900
time : 19756.667005300522


76000
time : 19782.919788122177


76100
time : 19808.968411445618


76200
time : 19834.954357862473


76300
time : 19860.94639968872


76400
time : 19886.92219567299


76500
time : 19912.863590478897


76600
time : 19938.825532913208


76700
time : 19964.90173316002


76800
time : 19990.861191034

98300
time : 25631.660868167877


98400
time : 25659.03279685974


98500
time : 25685.631297826767


98600
time : 25711.71838569641


98700
time : 25737.809194803238


98800
time : 25763.886259555817


98900
time : 25789.922317028046


99000
time : 25815.846841573715


99100
time : 25841.92089819908


99200
time : 25868.086919784546


99300
time : 25894.049552679062


99400
time : 25919.997198343277


99500
time : 25945.83819413185


99600
time : 25971.90566635132


99700
time : 25998.02289557457


99800
time : 26024.139189243317


99900
time : 26050.16264486313


100000
time : 26076.27669620514


100100
time : 26102.452878952026


100200
time : 26128.767018079758


100300
time : 26154.78479528427


100400
time : 26180.79897212982


100500
time : 26206.92809700966


100600
time : 26232.918914794922


100700
time : 26258.808399438858


100800
time : 26284.726055383682


100900
time : 26310.67395210266


101000
time : 26336.723295927048


101100
time : 26362.737698554993


101200
time : 

In [69]:
train_topgnr

,tags,id,plylst_title,songs,like_cnt,updt_date,most_appearing_gnr
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,GN0900
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,GN0100
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,GN0400
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,GN0100
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,GN0200
...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,GN1000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,GN1100
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,GN0100
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,GN1000


In [70]:
#dataframe으로 저장
train_topgnr.to_json('./train_topgnr.json', orient='table')

In [48]:
new_data = pd.read_json('./train_topgnr.json', orient='table')
new_data

,tags,id,plylst_title,songs,like_cnt,updt_date,most_appearing_gnr
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,GN0900
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,GN0100
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,GN0400
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,GN0100
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,GN0200
...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,GN1000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,GN1100
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,GN0100
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,GN1000


In [46]:
# song id별 장르 목록
song_gnr_map

,song_id,gnr_code
0,0,GN0900
1,1,GN1600
2,2,GN0900
3,3,GN1100
4,4,GN1800
...,...,...
802854,707984,GN2000
802855,707985,GN0900
802856,707986,GN0100
802857,707987,GN1800


In [47]:
#플레이리스트별 노래 목록
plylst_song_map

,id,songs
0,61281,525514
1,61281,129701
2,61281,383374
3,61281,562083
4,61281,297861
...,...,...
5285866,100389,111365
5285867,100389,51373
5285868,100389,640239
5285869,100389,13759


# 학습 방향..?
- ml, dl 사용을 위하여 train data를 어떻게 학습시킬건가 